In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from bert_serving.client import BertClient
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
from nltk.corpus import stopwords
from gensim.models import Word2Vec
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
import gensim
from gensim import corpora

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\603766\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('../Data/Product_Feelings.csv')

In [3]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
str_tweet=[str(tweet) for tweet in df['tweet_text']]
tweet_array = [clean(tweet).split() for tweet in str_tweet] 

In [5]:
bigram = gensim.models.Phrases(tweet_array, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tweet_array], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[tweet_array[0]]])

['wesley83', '3g', 'iphone', '3', 'hr', 'tweeting', 'riseaustin', 'dead', 'need', 'upgrade', 'plugin', 'station', 'sxsw']


In [6]:
sentences = df.tweet_text.values
new_sent=[]
for i in sentences:
    new_sent.append(str(i))
sentences = [sentence + " [SEP] [CLS]" for sentence in new_sent]
labels = df.is_there_an_emotion_directed_at_a_brand_or_product.values

In [7]:
tokenizer_1 = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts_1 = [tokenizer_1.tokenize(sent) for sent in sentences]
print(tokenized_texts_1[0])

['.', '@', 'wesley', '##8', '##3', 'i', 'have', 'a', '3', '##g', 'iphone', '.', 'after', '3', 'hr', '##s', 't', '##wee', '##ting', 'at', '#', 'rise', '_', 'austin', ',', 'it', 'was', 'dead', '!', 'i', 'need', 'to', 'upgrade', '.', 'plug', '##in', 'stations', 'at', '#', 's', '##x', '##sw', '.', '[SEP]', '[CLS]']


In [8]:
MAX_LEN = 128

In [9]:
input_ids_1 = [tokenizer_1.convert_tokens_to_ids(x) for x in tokenized_texts_1]

In [10]:
input_ids_1 = pad_sequences(input_ids_1, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [11]:
X_train_1, X_test_1,y_train_1,y_test_1 = train_test_split(input_ids_1,labels,test_size=.2)

In [12]:
X_train_1

array([[ 6203,  1055,  2595, ...,     0,     0,     0],
       [ 7158,  5743, 12927, ...,     0,     0,     0],
       [ 2012, 22878,  1010, ...,     0,     0,     0],
       ...,
       [19387,  1030,  5254, ...,     0,     0,     0],
       [ 2065,  1045,  2018, ...,     0,     0,     0],
       [ 3984,  1045,  2342, ...,     0,     0,     0]])

In [13]:
from sklearn.ensemble import RandomForestClassifier
inst_rf=RandomForestClassifier()
fit_rf=inst_rf.fit(X_train_1,y_train_1)
y_pred_1=fit_rf.predict(X_test_1)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_1,y_test_1)

C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5568993952721275

In [14]:
sentences = df.tweet_text.values
new_sent=[]
for i in sentences:
    new_sent.append(str(i))
sentences = ["[SEP]"+sentence +"[CLS]" for sentence in new_sent]
labels = df.is_there_an_emotion_directed_at_a_brand_or_product.values

In [28]:
tokenizer_2 = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)
tokenized_texts_2 = [tokenizer_2.tokenize(sent) for sent in sentences]
print(tokenized_texts_2[0])

['▁[', 's', 'ep', ']', '.', '@', 'we', 'sley', '83', '▁', 'i', '▁have', '▁a', '▁3', 'g', '▁', 'i', 'phone', '.', '▁after', '▁3', '▁', 'hr', 's', '▁tweet', 'ing', '▁at', '▁', '#', 'rise', '_', 'au', 'stin', ',', '▁it', '▁was', '▁dead', '!', '▁', 'i', '▁need', '▁to', '▁upgrade', '.', '▁plugin', '▁stations', '▁at', '▁', '#', 's', 'x', 's', 'w', '.', '[', 'cl', 's', ']']


In [ ]:
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

In [16]:
input_ids_2 = [tokenizer_2.convert_tokens_to_ids(x) for x in tokenized_texts_2]

In [17]:
input_ids_2 = pad_sequences(input_ids_2, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [18]:
X_train_2, X_test_2,y_train_2,y_test_2 = train_test_split(input_ids_2,labels,test_size=.2)

In [19]:
inst_rf=RandomForestClassifier()
fit_rf=inst_rf.fit(X_train_2,y_train_2)
y_pred_2=fit_rf.predict(X_test_2)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_2,y_test_2)

C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5585486531061022

In [20]:
tweets=df['tweet_text']

In [21]:
tweet_array=[]
for i in tweets:
    tweet_array.append(str(i))

In [22]:
len(tweet_array)

9093

In [23]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [24]:
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(tweet_array)
encoded_docs = tokenizer.texts_to_sequences(tweet_array)

In [25]:
input_ids_3 = pad_sequences(encoded_docs, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [26]:
X_train_3, X_test_3,y_train_3,y_test_3 = train_test_split(input_ids_3,labels,test_size=.2)

In [27]:
inst_rf=RandomForestClassifier()
fit_rf=inst_rf.fit(X_train_3,y_train_3)
y_pred_3=fit_rf.predict(X_test_3)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_3,y_test_3)

C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5678944474986256